# Brute Forcing Kryptos — An Attempt

Attempting to brute force the unsolved Kryptos cipher (K4 segment).

Using Python because I'm rusty as fuck with C at the moment. Plus I've recently used a lot of Python with GPU acceleration in my [CS thesis](https://github.com/vinivosh/ufu-tcc2), so it'll be very useful here and won't require me to learn new stuff — always a great bonus for a lazy lad.

I'll try using [Numba](https://numba.pydata.org/)'s [jit compiler](https://numba.readthedocs.io/en/stable/user/jit.html) to accelerate the process this as much as possible. Preferably even Numba's [CUDA support](https://numba.readthedocs.io/en/stable/cuda/overview.html) if it works.

# The Segments

Of note: **K2** here is the corrected version. For the version in the actual sculpture, with the omission error, just substitute the `ESWJL` near the end by `EWJL`

In [40]:
K1 = '''EMUFPHZLRFAXYUSDJKZLDKRNSHGNFIVJ
YQTQUXQBQVYUVLLTREVJYQTMKYRDMFD'''

K2 = '''VFPJUDEEHZWETZYVGWHKKQETGFQJNCE
GGWHKK?DQMCPFQZDQMMIAGPFXHQRLG
TIMVMZJANQLVKQEDAGDVFRPJUNGEUNA
QZGZLECGYUXUEENJTBJLBQCRTBJDFHRR
YIZETKZEMVDUFKSJHKFWHKUWQLSZFTI
HHDDDUVH?DWKBFUFPWNTDFIYCUQZERE
EVLDKFEZMOQQJLTTUGSYQPFEUNLAVIDX
FLGGTEZ?FKZBSFDQVGOGIPUFXHHDRKF
FHQNTGPUAECNUVPDJMQCLQUMUNEDFQ
ELZZVRRGKFFVOEEXBDMVPNFQXEZLGRE
DNQFMPNZGLFLPMRJQYALMGNUVPDXVKP
DQUMEBEDMHDAFMJGZNUPLGESWJLLAETG'''

K3 = '''ENDYAHROHNLSRHEOCPTEOIBIDYSHNAIA
CHTNREYULDSLLSLLNOHSNOSMRWXMNE
TPRNGATIHNRARPESLNNELEBLPIIACAE
WMTWNDITEENRAHCTENEUDRETNHAEOE
TFOLSEDTIWENHAEIOYTEYQHEENCTAYCR
EIFTBRSPAMHHEWENATAMATEGYEERLB
TEEFOASFIOTUETUAEOTOARMAEERTNRTI
BSEDDNIAAHTTMSTEWPIEROAGRIEWFEB
AECTDDHILCEIHSITEGOEAOSDDRYDLORIT
RKLMLEHAGTDHARDPNEOHMGFMFEUHE
ECDMRIPFEIMEHNLSSTTRTVDOHW?'''

K4 = '''OBKR
UOXOGHULBSOLIFBBWFLRVQQPRNGKSSO
TWTQSJQSSEKZZWATJKLUDIAWINFBNYP
VTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR'''

# print(f'K1:\n{K1}\n\nK2:\n{K2}\n\nK3:\n{K3}\n\nK4:\n{K4}\n\n')

# Dictionary and Evaluation

How to know if the decrypted message candidates make sense and are not just random gibberish? Using some known patterns in the english language!

> I got this idea from [this great experiment](https://github.com/desgeeko/kryptos/blob/main/Kryptos.ipynb), so all credit goes to GitHub user **[desgeeko](https://github.com/desgeeko)**.

## Reading Files (Google Web Trillion Word Corpus)

In [41]:
import os
import time
from urllib.request import urlopen

import numpy as np
import pandas as pd
# from numba import jit

def download_if_needed(file_path, url):
    '''Downloads file from `url`, if it doesn't already exists in `filePath`'''

    if os.path.exists(file_path): return

    with urlopen(url) as f:
        html = f.read().decode('utf-8')
    with open(file_path, 'w') as f:
        f.write(html)

file_count_1w = 'count_1w.txt'
file_count_2l = 'count_2l.txt'
url_base = 'https://norvig.com/ngrams/'

download_if_needed(file_count_1w, url_base + file_count_1w)
download_if_needed(file_count_2l, url_base + file_count_2l)

# Reading ngrams files
with open(file_count_1w, 'r') as f:
    count_1w = pd.read_csv(f, names=['word', 'count'], sep='\t')
with open(file_count_2l, 'r') as f:
    count_2l = pd.read_csv(f, names=['bigram', 'count'], sep='\t')

# Normalizing bigram count, to help calculations
minBigramCount = count_2l['count'].min()
count_2l['count'] = (count_2l['count'] / minBigramCount).astype(np.uint32)

print(count_1w)
print(count_2l)

# Converting count_2l to a dictionary
count_2l = {bigram.upper(): count for (bigram, count) in count_2l.to_numpy()}
# print(count_2l)

           word        count
0           the  23135851162
1            of  13151942776
2           and  12997637966
3            to  12136980858
4             a   9081174698
...         ...          ...
333328    gooek        12711
333329   gooddg        12711
333330  gooblle        12711
333331   gollgo        12711
333332    golgw        12711

[333333 rows x 2 columns]
    bigram  count
0       in  47154
1       th  46594
2       er  41698
3       re  38010
4       he  37250
..     ...    ...
671     qy      2
672     zq      2
673     jx      1
674     qz      1
675     jq      1

[676 rows x 2 columns]


## Evaluation Function

**[TODO: add a good explanation of the function here]**

### Expected Scores

Testing with some example text and using the default function arguments, the results below were found.

| **Type of Text**      | **Sample Size** | **Average Score** |
|-----------------------|-----------------|-------------------|
| English               | 9               | 17,605            |
| Japanese Romaji       | 3               | 11,019            |
| Other Languages       | 4               | 16,153            |
| *All of the above*    | 16              | 16,007            |
| Random Chars (A to Z) | 10              | 3,464             |

English text was a mixture of classic texts, popular sayings and a sample phrase I wrote. Japanese Romaji were from Ichiko Aoba's lyrics. Other languages were translations the english sample phrase into spanish, french, dutch and brazilian portuguese.

In [42]:
def is_it_english(text:str, use_geo_mean=False, ignore_invalid_chars=True):
    '''Calculates and returns an integer score that represents how high is the chance of the string passed in `text` being a text in english. The higher the score, the higher the chance

    Could range from 1 to 47,154, but usually texts in a natural language score an average of 16,007 while random strings of letters A to Z score an average of 3,464'''

    bigrams = [text[i:i+2].upper() for i in range(len(text) - 1)]
    # print(bigrams)
    total = 0
    bigramsChecked = 0

    for bigram in bigrams:
        score = count_2l.get(bigram, None)
        # print(score)
        if score is None: continue
        bigramsChecked += 1

        if use_geo_mean: total += np.log(score)
        else: total += score

    if use_geo_mean: return round(np.exp(total / (bigramsChecked if ignore_invalid_chars else len(text))))
    return round(total / (bigramsChecked if ignore_invalid_chars else len(text)))

testTexts = [
    'TEST PHRASE THAT IS COMPLETELY WRITTEN IN ENGLISH LETS SEE IF THIS GOES EXACTLY AS I HAVE PLANNED',
    'A CAT NAP / A DAY LATE AND A DOLLAR SHORT / LOVE BIRDS / A LOT ON ONES PLATE / A BITE AT THE CHERRY / RAINING CATS AND DOGS',
    'The Lord is my shepherd; I shall not want. / He maketh me to lie down in green pastures: he leadeth me',
    'I MET A TRAVELLER from an antique land / Who said: Two vast and trunkless legs of stone / Stand in the desert.',
    'And yet I say unto you, That even Solomon in all his glory was not arrayed like one of these.',
    'I wish I loved the Human Race; / I wish I loved its silly face; / I wish I liked the way it walks',
    'Mondays child is fair of face, / Tuesdays child is full of grace, / Wednesdays child is full of woe',
    'A grasshopper spent the summer hopping about in the sun and singing to his hearts content',
    'One day the Hare laughed at the short feet and slow speed of the Tortoise. The Tortoise replied: / "You may',
    'When I was a child, I spake as a child, I understood as a child, I thought as a child: but when I became',

    'VOANQOYESPYIZAITGKLJCEVZLKXGPZCMGEWDWXOHPOQBMVPGPHQIDBEWBUBALFFCOVWVHVIKAFLVEDIGIJBPMDQWYEMALSJT',
    'JFGEHMBDMFXRKUVBQSBPJNZAAUVLSZWGLAYSLWHLGSQRADVBYEMJYPEIYRFXSLRKYBUXODJPCDLJYVQZGMKTOEQZILSAIVYR',
    'GKJJGBTZULSEGNXAJCQPEQCFCMIGIEBXEQDVJZTQSUWPUFMVEPDITAEOOHRCCGSJSCUZPEVGXZHNNIJVGTWRUVPJZRFWSIVH',
    'CWHPWFBKRDNUQMYMDDRPXPAJVXPTDAYWUOHRRAGJFVIBRCRBCQFUTUFDCZDTDJUCWDQYBJYBMIBMPQWVKPLSSYKKKROEEBQT',
    'QNBNJHKDWFYSNGUSIMFTSDGQWNZWNDLAWXTTSXZMMKGAXDQODQNWXBPOOJYHCKBJZGBQJSAHSMCFNSXTDEDSERZECWWJGAVY',
    'KGGJHZNRXUACPTLXOWRACJXLNZFYDKDTYZXAQEVKPZNWUYFWGLUNMKULBBIBYCVERAWXUYDOVANGAMEKZAROVZWEAZVDRMVY',
    'CLPCODRAVOWNSFWMPARDZYENCCPYBLIFIXHVWNFTYQHMPNWQRYKWUGIGHXQENQIOAKFGFCYMMILOPVPEZHKUAZXKDXOZKPKS',
    'IXYIIXRRRUGWJCXIGOHCWDUKBIYYWBLUVVYEWVGOPTBEASNUKTHELYWIRUJNJYIFXCMKRBIOQSDOBTYSTBYWHKFOEKBFVEET',
    'HIJUVQATUXCSCIMDMYOOERZHHOHBIYNGRIFAAUSLVFYGVFATTHXDQPGQIDIQPLXJHTYITAAYWKXEDINKODERMCKZOOGWFJAB',
    'NJGCUTBSIRCFGVPVDDLODXSAEEGKAJDSQVWHFXATKCBOAHTVWYVGGYIXNRGHDCPZIQNRQCVAVPYHTNXHYDMIDTDENGYUJPLB',
]

runs = 333340 // len(testTexts)
timings = []

for text in testTexts:
    for runIdx in range(runs):
        text = text.upper()
        startTime = time.perf_counter_ns()
        score = is_it_english(text)
        timings.append(time.perf_counter_ns() - startTime)
    print(f'"{text}"\n{score}\n')

timings = np.array(timings, np.int64)
print(f'Total time to score {runs * len(testTexts)} cipher texts = {timings.sum() * 1e-6} ms')
print(f'Avg time to score a cipher text = {timings.mean()} \u00B1 {timings.std()} ns')

# Testing some bigger and classic english texts
testTexts = [
    #
]

"TEST PHRASE THAT IS COMPLETELY WRITTEN IN ENGLISH LETS SEE IF THIS GOES EXACTLY AS I HAVE PLANNED"
15967

"A CAT NAP / A DAY LATE AND A DOLLAR SHORT / LOVE BIRDS / A LOT ON ONES PLATE / A BITE AT THE CHERRY / RAINING CATS AND DOGS"
18106

"THE LORD IS MY SHEPHERD; I SHALL NOT WANT. / HE MAKETH ME TO LIE DOWN IN GREEN PASTURES: HE LEADETH ME"
19337

"I MET A TRAVELLER FROM AN ANTIQUE LAND / WHO SAID: TWO VAST AND TRUNKLESS LEGS OF STONE / STAND IN THE DESERT."
18180

"AND YET I SAY UNTO YOU, THAT EVEN SOLOMON IN ALL HIS GLORY WAS NOT ARRAYED LIKE ONE OF THESE."
17357

"I WISH I LOVED THE HUMAN RACE; / I WISH I LOVED ITS SILLY FACE; / I WISH I LIKED THE WAY IT WALKS"
14081

"MONDAYS CHILD IS FAIR OF FACE, / TUESDAYS CHILD IS FULL OF GRACE, / WEDNESDAYS CHILD IS FULL OF WOE"
10405

"A GRASSHOPPER SPENT THE SUMMER HOPPING ABOUT IN THE SUN AND SINGING TO HIS HEARTS CONTENT"
19851

"ONE DAY THE HARE LAUGHED AT THE SHORT FEET AND SLOW SPEED OF THE TORTOISE. THE TORTOISE REPLIED: / "YOU MAY"


### Benchmarking

The evaluation function was benchmarked and yielded the following results

Using arithmetic mean (regular average operation):
```
Total time to score 333340 cipher texts = 3977.749178 ms
Avg time to score a cipher text = 11933.008873822524 ± 2892.99181172723 ns

```

Using geometric mean:
```
Total time to score 333340 cipher texts = 19401.050930999998 ms
Avg time to score a cipher text = 58201.988753224934 ± 15424.9984648464 ns

```

# Brute Forcing K1